# Evaluación de Modelos de Lenguaje - OWASP Generative API
Este notebook está diseñado para evaluar el rendimiento de tres modelos de lenguaje en el contexto de preguntas y respuestas sobre OWASP Top 10. Los modelos evaluados son:

1. bloom
2. gpt-neo-2.7B
3. Flan-T5-Base

Se utilizarán métricas avanzadas como Exact Match, BLEU y ROUGE para evaluar la calidad de las respuestas generadas. Además, se medirá la latencia y el tiempo total de evaluación.

In [1]:
# Instalación de dependencias necesarias
!pip install transformers torch tqdm nltk rouge-score python-dotenv


## Configuración inicial del notebook
Configuramos el entorno para que soporte la evaluación de los modelos. Esto incluye la carga de librerías y la configuración del dataset de prueba.

In [2]:
import time
import json
import os
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from dotenv import load_dotenv

In [3]:
load_dotenv('.env')

hf_token = os.environ['HF_TOKEN']

Configuramos el token para Hugging Face.



In [4]:
from huggingface_hub import login

hf_token = os.environ['HF_TOKEN']
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Subida del dataset de prueba
Subimos el archivo `test_questions.json` que contiene las preguntas y respuestas esperadas.

In [6]:
from google.colab import files

uploaded = files.upload()
dataset_file = 'test_questions.json'

with open(dataset_file, 'r') as f:
    test_questions = json.load(f)

Saving test_questions.json to test_questions (1).json


In [7]:
dataset_file_2 = 'test_questions_categories.json'

uploaded = files.upload()
dataset_file = 'test_questions_categories.json'

with open(dataset_file_2, 'r') as f:
    test_questions_categories = json.load(f)

print(f'Dataset cargado con {len(test_questions_categories)} preguntas.')

Saving test_questions_categories.json to test_questions_categories.json
Dataset cargado con 3 preguntas.


## Funciones para la evaluación de modelos
Se define una función `evaluate_model` para realizar la evaluación de cada modelo, calculando las métricas Exact Match, BLEU y ROUGE, además de medir la latencia de las respuestas.

In [31]:
import time
from transformers import pipeline
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

def evaluate_text_generation_model(model_name, test_questions):
    """
    Evalúa un modelo de generación de texto en base a un conjunto de prompts con contexto y respuestas esperadas.
    """
    print(f'\nEvaluando modelo: {model_name}')
    start_time = time.time()

    # Cargar el modelo
    text_gen_pipeline = pipeline('text-generation', model=model_name, tokenizer=model_name)

    # Inicializar métricas
    total_bleu = 0
    total_rouge1 = 0
    total_rougeL = 0
    latencies = []

    # Inicializar evaluador ROUGE
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    results = []

    for test in test_questions:
        question = test['question']
        context = test['expected']
        prompt = f"{question}"

        # Medir latencia
        start = time.time()
        result = text_gen_pipeline(
            prompt,
            max_length=80,
            do_sample=True,
            temperature=0.5,
            top_k=50,
            num_return_sequences=1
        )
        latency = time.time() - start
        latencies.append(latency)

        generated = result[0]['generated_text'] if isinstance(result, list) else result['generated_text']
        print(f'\nPregunta: {question}')
        print(f'Respuesta generada: {generated}')
        print(f'Respuesta esperada: {context}')
        print(f'Latencia: {latency:.4f} segundos')

        # BLEU Score
        bleu_score = sentence_bleu([context.split()], generated.split())
        total_bleu += bleu_score

        # ROUGE Scores
        rouge_scores = rouge.score(context, generated)
        total_rouge1 += rouge_scores['rouge1'].fmeasure
        total_rougeL += rouge_scores['rougeL'].fmeasure

        # Almacenar resultados individuales
        results.append({
            'question': question,
            'expected': context,
            'generated': generated,
            'latency': latency,
            'bleu': bleu_score,
            'rouge1': rouge_scores['rouge1'].fmeasure,
            'rougeL': rouge_scores['rougeL'].fmeasure
        })

    # Promedios
    avg_bleu = total_bleu / len(test_questions)
    avg_rouge1 = total_rouge1 / len(test_questions)
    avg_rougeL = total_rougeL / len(test_questions)
    avg_latency = sum(latencies) / len(latencies)
    total_time = time.time() - start_time

    # Resultados finales
    final_results = {
        'model_name': model_name,
        'avg_bleu': avg_bleu,
        'avg_rouge1': avg_rouge1,
        'avg_rougeL': avg_rougeL,
        'avg_latency': avg_latency,
        'total_time': total_time,
        'details': results
    }

    print(f'\n--- Resultados del modelo {model_name} ---')
    print(f'BLEU Promedio: {avg_bleu:.4f}')
    print(f'ROUGE-1 Promedio: {avg_rouge1:.4f}')
    print(f'ROUGE-L Promedio: {avg_rougeL:.4f}')
    print(f'Latencia Promedio: {avg_latency:.4f} segundos')
    print(f'Tiempo Total: {total_time:.2f} segundos\n')

    return final_results


Complementario a esto, se realiza una función adicional `evaluate_model_categories` para probar el dataset complementario el cual evalúa 3 categorías: robustez, respuestas abiertas, y multilingüismo por separado.

In [32]:
def evaluate_models_with_categories(model_name, test_categories):
    """
    Evalúa un modelo de generación de texto en base a categorías de preguntas.
    """
    print(f'\nEvaluando modelo: {model_name} por categorías')
    all_results = {}

    for category in test_categories:
        category_name = category['category']
        print(f'\nCategoría: {category_name}')

        # Evaluar cada categoría
        category_results = evaluate_text_generation_model(model_name, category['questions'])
        all_results[category_name] = category_results

    return all_results

### Evaluación de los modelos seleccionados
Se evalúan los siguientes modelos:

1. bloom
2. gpt-neo-2.7B
3. Flan-T5-Base

In [33]:
models_to_test = [
    'bigscience/bloom-560m',
    'PlanTL-GOB-ES/gpt2-base-bne',
    'google/flan-t5-base'
]

all_results = []

for model in models_to_test:
    model_results = evaluate_text_generation_model(model, test_questions)
    all_results.append(model_results)


Evaluando modelo: bigscience/bloom-560m


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Pregunta: ¿Qué es una inyección SQL?
Respuesta generada: ¿Qué es una inyección SQL? ¿Es una inyección SQL? – Stack Overflow en español
Una inyección SQL es una forma de hacer que un programa que se ejecuta en un servidor web se ejecute en otro servidor web. Este tipo de inyección es más comúnmente utilizada para hacer que un programa se ejecute en el servidor web, pero también se puede utilizar para hacer que un programa se ejecute en
Respuesta esperada: Una técnica de ataque que explota fallos en la validación de entradas.
Latencia: 1.6613 segundos

Pregunta: ¿Cómo prevenir XSS?
Respuesta generada: ¿Cómo prevenir XSS? ¿Cuales son los mejores navegadores? ¿Cuales son los mejores antivirus? ¿Cuales son los mejores firewall? ¿Cuales son los mejores navegadores web? ¿Cuales son los mejores antivirus? ¿Cuales son los mejores firewall? ¿Cuales son los mejores navegadores web? ¿Cuales son los mejores antivirus? ¿Cuales
Respuesta esperada: Validando entradas y escapando caracteres especiales

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Pregunta: ¿Qué es una inyección SQL?
Respuesta generada: ¿Qué es una inyección SQL? 
Respuesta esperada: Una técnica de ataque que explota fallos en la validación de entradas.
Latencia: 0.0235 segundos

Pregunta: ¿Cómo prevenir XSS?
Respuesta generada: ¿Cómo prevenir XSS? - 
Respuesta esperada: Validando entradas y escapando caracteres especiales.
Latencia: 0.0275 segundos

Pregunta: ¿Qué es una vulnerabilidad CSRF?
Respuesta generada: ¿Qué es una vulnerabilidad CSRF? 
Respuesta esperada: Un ataque que permite realizar acciones no autorizadas en nombre de un usuario autenticado.
Latencia: 0.0189 segundos

--- Resultados del modelo PlanTL-GOB-ES/gpt2-base-bne ---
BLEU Promedio: 0.0000
ROUGE-1 Promedio: 0.0667
ROUGE-L Promedio: 0.0667
Latencia Promedio: 0.0233 segundos
Tiempo Total: 1.71 segundos


Evaluando modelo: google/flan-t5-base


Device set to use cuda:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'Mar


Pregunta: ¿Qué es una inyección SQL?
Respuesta generada: ¿Qué es una inyección SQL?f) a g) a f) a g) a h) a i ) a j ) )
Respuesta esperada: Una técnica de ataque que explota fallos en la validación de entradas.
Latencia: 1.1585 segundos

Pregunta: ¿Cómo prevenir XSS?
Respuesta generada: ¿Cómo prevenir XSS?
Respuesta esperada: Validando entradas y escapando caracteres especiales.
Latencia: 0.1546 segundos

Pregunta: ¿Qué es una vulnerabilidad CSRF?
Respuesta generada: ¿Qué es una vulnerabilidad CSRF?
Respuesta esperada: Un ataque que permite realizar acciones no autorizadas en nombre de un usuario autenticado.
Latencia: 0.1411 segundos

--- Resultados del modelo google/flan-t5-base ---
BLEU Promedio: 0.0000
ROUGE-1 Promedio: 0.0404
ROUGE-L Promedio: 0.0404
Latencia Promedio: 0.4847 segundos
Tiempo Total: 2.64 segundos



In [34]:
models_to_test = [
    'bigscience/bloom-560m',
    'PlanTL-GOB-ES/gpt2-base-bne',
    'google/flan-t5-base'
]

all_results_categories = []

for model in models_to_test:
    model_results = evaluate_models_with_categories(model, test_questions_categories)
    all_results_categories.append(model_results)


Evaluando modelo: bigscience/bloom-560m por categorías

Categoría: robustez

Evaluando modelo: bigscience/bloom-560m


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Pregunta: Kómo prebenir XSS?
Respuesta generada: Kómo prebenir XSS? - Stack Overflow en español
Estoy usando un plugin para que mis scripts de php se actualicen en caso de que uno de mis scripts se ejecute en un proceso de una página web. El problema es que al ejecutar el script, este se ejecuta en un proceso diferente al que está programado en la página web. Por ejemplo, si ejecuto el
Respuesta esperada: Validando entradas y escapando caracteres especiales.
Latencia: 1.5769 segundos

Pregunta: Q es un CSRF?
Respuesta generada: Q es un CSRF? ¿Son válidas las cookies para la navegación y para el registro de la sesión?
No. El CSRF no es necesario para la navegación y para el registro de la sesión y, por lo tanto, no afecta a la funcionalidad de los sitios web ni a la seguridad de las transacciones en línea.
Respuesta esperada: Un ataque que permite realizar acciones no autorizadas en nombre de un usuario autenticado.
Latencia: 1.3866 segundos

Pregunta: Queé es SQL injection?
Respuesta 

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Pregunta: ¿Por qué es importante validar entradas en una aplicación web?
Respuesta generada: ¿Por qué es importante validar entradas en una aplicación web? ¿El uso de un certificado SSL es importante? ¿El uso de un certificado TLS es importante? ¿El uso de un certificado SSL puede hacer que una aplicación web sea más segura? ¿Cómo validar las entradas en una aplicación web? ¿El uso de un certificado SSL es importante? ¿El uso de un certificado TLS es importante? ¿El uso
Respuesta esperada: Validar entradas previene ataques como inyecciones SQL y XSS, mejorando la seguridad de la aplicación.
Latencia: 1.5716 segundos

Pregunta: ¿Cuáles son los riesgos de no validar entradas?
Respuesta generada: ¿Cuáles son los riesgos de no validar entradas? ¿Cómo comprobar si una entrada ha sido validada? ¿Debería validar entradas de modo que no se produzca un error?
Respuesta esperada: La aplicación puede ser vulnerable a ataques como SQL Injection, XSS y ejecución de código remoto.
Latencia: 0.5981 

...


Pregunta: Kómo prebenir XSS?
Respuesta generada: Kómo prebenir XSS?
Respuesta esperada: Validando entradas y escapando caracteres especiales.
Latencia: 0.1637 segundos

Pregunta: Q es un CSRF?
Respuesta generada: Q es un CSRF?ople for a cause
Respuesta esperada: Un ataque que permite realizar acciones no autorizadas en nombre de un usuario autenticado.
Latencia: 0.1970 segundos

Pregunta: Queé es SQL injection?
Respuesta generada: Queé es SQL injection?
Respuesta esperada: Un ataque que explota fallos en la validación de entradas para ejecutar consultas maliciosas en una base de datos.
Latencia: 0.1285 segundos

Pregunta: Comóo evitar SQL INJECTION?
Respuesta generada: Comóo evitar SQL INJECTION?
Respuesta esperada: Usando sentencias preparadas y validando entradas del usuario.
Latencia: 0.1734 segundos

--- Resultados del modelo google/flan-t5-base ---
BLEU Promedio: 0.0000
ROUGE-1 Promedio: 0.0436
ROUGE-L Promedio: 0.0436
Latencia Promedio: 0.1657 segundos
Tiempo Total: 1.88 segundo

Device set to use cuda:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'Mar


Pregunta: ¿Por qué es importante validar entradas en una aplicación web?
Respuesta generada: ¿Por qué es importante validar entradas en una aplicación web?
Respuesta esperada: Validar entradas previene ataques como inyecciones SQL y XSS, mejorando la seguridad de la aplicación.
Latencia: 0.1982 segundos

Pregunta: ¿Cuáles son los riesgos de no validar entradas?
Respuesta generada: ¿Cuáles son los riesgos de no validar entradas?
Respuesta esperada: La aplicación puede ser vulnerable a ataques como SQL Injection, XSS y ejecución de código remoto.
Latencia: 0.5553 segundos

Pregunta: Explique cómo funciona una vulnerabilidad de tipo CSRF.
Respuesta generada: Explique cómo funciona una vulnerabilidad de tipo CSRF.
Respuesta esperada: CSRF ocurre cuando un atacante induce a un usuario autenticado a ejecutar acciones no deseadas en un sistema donde está logueado, como transferencias de dinero o cambios de configuración.
Latencia: 0.3118 segundos

--- Resultados del modelo google/flan-t5-bas

Device set to use cuda:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'Mar


Pregunta: What is SQL Injection?
Respuesta generada: What is SQL Injection?
Respuesta esperada: A technique that exploits input validation flaws to execute malicious queries on a database.
Latencia: 0.1066 segundos

Pregunta: Como prevenir XSS?
Respuesta generada: Como prevenir XSS?
Respuesta esperada: Validando entradas e escapando caracteres especiais.
Latencia: 0.1537 segundos

Pregunta: Qué es un ataque de inyección SQL?
Respuesta generada: Qué es un ataque de inyección SQL?
Respuesta esperada: Un ataque que explota fallos en la validación de entradas para ejecutar consultas maliciosas en una base de datos.
Latencia: 0.3436 segundos

Pregunta: How to prevent CSRF?
Respuesta generada: How to prevent CSRF?d by avoiding exposure to the elements.
Respuesta esperada: Using CSRF tokens and verifying their validity for each user action.
Latencia: 0.2955 segundos

--- Resultados del modelo google/flan-t5-base ---
BLEU Promedio: 0.0000
ROUGE-1 Promedio: 0.0942
ROUGE-L Promedio: 0.0763
Late

### Guardar resultados en un informe
Los resultados de la evaluación se guardan en un archivo JSON llamado `evaluation_results.json` y `evaluation_results_categories.json` respectivamente.

In [35]:
output_file = 'evaluation_results.json'
with open(output_file, 'w') as f:
    json.dump(all_results, f, indent=4)

print(f'Resultados guardados en {output_file}')

Resultados guardados en evaluation_results.json


In [36]:
output_file_categories = 'evaluation_results_categories.json'
with open("evaluation_results_categories.json", "w") as f:
    json.dump(all_results_categories, f, indent=4)

print(f'Resultados guardados en {output_file}')


Resultados guardados en evaluation_results.json
